In [1]:
#pip install --upgrade pip
#pip install beautifulsoup4

In [2]:
import requests
from bs4 import BeautifulSoup
import threading

In [3]:
BASE_ADDRESS_URI = "https://animelayer.ru"
THREAD_MAX = 25

In [4]:
class TorrentData:
    def init(self):
        self.guid = ""
        self.name = ""
    def __str__(self):
        return f'{self.guid}: {self.name}'
    def __repr__(self):
        return self.__str__()

In [5]:
def get_identifiers_from_page(url):
    response = requests.get(url)
    if response.status_code == 200:
        html_doc = response.text
        soup = BeautifulSoup(html_doc, 'html.parser')
        div_values = []
        for div in soup.find_all('h3', {'class':'h2 m0'}):
            style = div.attrs['style']
            if(style.find("view-transition-name: title-") > -1):
                data = TorrentData()
                data.guid = style.removeprefix("view-transition-name: title-")
                data.name = div.getText().replace('\xa0', ' ')
                div_values.append(data)
        return div_values
    else:
        print(f'[{response.status_code}] {response.text}')
        return None

In [ ]:
def collectBaseItemsFromAddress(ADDRESS):
    result = {}
    pages = {}
    def get_identifiers_from_page_async(iPage):
        page = get_identifiers_from_page(f"{ADDRESS}/?page={iPage}")
        result[iPage] = not (page == None or len(page) < 1)
        if(result[iPage] == True):
            pages[iPage] = page

    iPage = 1
    while True:
        threads = []
        for iThread in range(THREAD_MAX):
                t = threading.Thread(target=get_identifiers_from_page_async, args=([iPage + iThread]))
                t.start()
                threads.append(t)

        for t in threads:
            t.join()

        if(all(result.values()) == False):
            break

        iPage = iPage + THREAD_MAX

In [6]:
pages = collectBaseItemsFromAddress(f"{BASE_ADDRESS_URI}/torrents/anime/")